# (Optional) Colab Setup
If you aren't using Colab, you can delete the following code cell. This is just to help students with mounting to Google Drive to access the other .py files and downloading the data, which is a little trickier on Colab than on your local machine using Jupyter. 

In [1]:
# you will be prompted with a window asking to grant permissions
# from google.colab import drive
# drive.mount("/content/drive")

In [2]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
# import os
# datadir = "/content/assignment1"
# if not os.path.exists(datadir):
#   !ln -s "/content/drive/My Drive/YOUR PATH HERE/assignment1/" $datadir
# os.chdir(datadir)
# !pwd

In [3]:
# downloading Fashion-MNIST
# import os
# os.chdir(os.path.join(datadir,"fashion-mnist/"))
# !chmod +x ./get_data.sh
# !./get_data.sh
# os.chdir(datadir)

# Imports

In [4]:
import random
import numpy as np
from data_process import get_FASHION_data, get_RICE_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline
np.random.seed(441)
# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [5]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [6]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Loading Rice

In [7]:
# loads train / test / val splits of 80%, 20%, 20% 
data = get_RICE_data()
X_train_RICE, y_train_RICE = data['X_train'], data['y_train']
X_val_RICE, y_val_RICE = data['X_val'], data['y_val']
X_test_RICE, y_test_RICE = data['X_test'], data['y_test']
n_class_RICE = len(np.unique(y_test_RICE))

print("Number of train samples: ", X_train_RICE.shape[0])
print("Number of val samples: ", X_val_RICE.shape[0])
print("Number of test samples: ", X_test_RICE.shape[0])

Number of train samples:  10911
Number of val samples:  3637
Number of test samples:  3637


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [8]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
### Learning rate
The learning rate controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. Here is a guide to help you find a right learning rate: 
- Try values ranging from 5.0 to 0.0005 to see the impact on model accuracy. 
- If the accuracy fluctuates a lot or diverges, the learning rate is too high. Try decreasing it by a factor of 10 (e.g. from 0.5 to 0.05). 
- If the accuracy is changing very slowly, the learning rate may be too low. Try increasing it by a factor of 10.
- You can also try adding a learning rate decay to slowly reduce the learning rate over each training epoch. For example, multiply the learning rate by 0.95 after each epoch.
- Plot training and validation accuracy over epochs for different learning rates. This will help you visualize the impact of the learning rate.
- [Here](https://towardsdatascience.com/https-medium-com-dashingaditya-rakhecha-understanding-learning-rate-dd5da26bb6de) is a detailed guide to learning rate.

### Number of Epochs
An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on Fashion-MNIST

In [68]:
lr = 0.02
n_epochs = 40

percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

Epoch 0, error: 0.3746, lr: 0.0194
Epoch 1, error: 0.32554, lr: 0.018818
Epoch 2, error: 0.31088, lr: 0.01825346
Epoch 3, error: 0.30516, lr: 0.017705856199999998
Epoch 4, error: 0.29432, lr: 0.017174680513999997
Epoch 5, error: 0.29318, lr: 0.016659440098579996
Epoch 6, error: 0.28778, lr: 0.016159656895622595
Epoch 7, error: 0.28714, lr: 0.015674867188753917
Epoch 8, error: 0.28484, lr: 0.0152046211730913
Epoch 9, error: 0.27992, lr: 0.014748482537898561
Epoch 10, error: 0.27782, lr: 0.014306028061761605
Epoch 11, error: 0.2786, lr: 0.013876847219908756
Epoch 12, error: 0.275, lr: 0.013460541803311493
Epoch 13, error: 0.27386, lr: 0.013056725549212148
Epoch 14, error: 0.27076, lr: 0.012665023782735783
Epoch 15, error: 0.26998, lr: 0.01228507306925371
Epoch 16, error: 0.27026, lr: 0.011916520877176097
Epoch 17, error: 0.26804, lr: 0.011559025250860814
Epoch 18, error: 0.26702, lr: 0.011212254493334989
Epoch 19, error: 0.2675, lr: 0.010875886858534939
Epoch 20, error: 0.26234, lr: 0.01

In [69]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 80.166000


### Validate Perceptron on Fashion-MNIST

In [36]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 80.170000


### Test Perceptron on Fashion-MNIST

In [37]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 79.380000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [26]:
# output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

## Train Perceptron on Rice

In [64]:
lr = 0.0001
n_epochs = 15

percept_RICE = Perceptron(n_class_RICE, lr, n_epochs)
percept_RICE.train(X_train_RICE, y_train_RICE)

Epoch 0, error: 0.2803592704610027, lr: 0.0001
Epoch 1, error: 0.26047108422692694, lr: 0.0001
Epoch 2, error: 0.23719182476399964, lr: 0.0001
Epoch 3, error: 0.18458436440289616, lr: 0.0001
Epoch 4, error: 0.14737420951333516, lr: 0.0001
Epoch 5, error: 0.0959582073137201, lr: 0.0001
Epoch 6, error: 0.08761800018330125, lr: 0.0001
Epoch 7, error: 0.08816790395014205, lr: 0.0001
Epoch 8, error: 0.05902300430757951, lr: 0.0001
Epoch 9, error: 0.057098341123636695, lr: 0.0001
Epoch 10, error: 0.04967464027128586, lr: 0.0001
Epoch 11, error: 0.047200073320502245, lr: 0.0001
Epoch 12, error: 0.03702685363394739, lr: 0.0001
Epoch 13, error: 0.038951516817890205, lr: 0.0001
Epoch 14, error: 0.032535972871414166, lr: 0.0001


In [65]:
pred_percept = percept_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))

The training accuracy is given by: 99.193474


### Validate Perceptron on Rice

In [66]:
pred_percept = percept_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))

The validation accuracy is given by: 99.257630


### Test Perceptron on Rice

In [67]:
pred_percept = percept_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))

The testing accuracy is given by: 99.120154


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on Fashion-MNIST

In [117]:
lr = 0.001
n_epochs = 30
reg_const = 2

svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const)
svm_fashion.train(X_train_fashion, y_train_fashion)

epoch 1 / 30, error: 0.93736
epoch 2 / 30, error: 0.22529999999999994
epoch 3 / 30, error: 0.21267999999999998
epoch 4 / 30, error: 0.20320000000000005
epoch 5 / 30, error: 0.19845999999999997
epoch 6 / 30, error: 0.19894
epoch 7 / 30, error: 0.20208000000000004
epoch 8 / 30, error: 0.20409999999999995
epoch 9 / 30, error: 0.19652000000000003
epoch 10 / 30, error: 0.20482
epoch 11 / 30, error: 0.19940000000000002
epoch 12 / 30, error: 0.20016
epoch 13 / 30, error: 0.20182
epoch 14 / 30, error: 0.18676000000000004
epoch 15 / 30, error: 0.19994
epoch 16 / 30, error: 0.20484000000000002
epoch 17 / 30, error: 0.20132000000000005
epoch 18 / 30, error: 0.19976000000000005
epoch 19 / 30, error: 0.1975
epoch 20 / 30, error: 0.19001999999999997
epoch 21 / 30, error: 0.20143999999999995
epoch 22 / 30, error: 0.19613999999999998
epoch 23 / 30, error: 0.19306
epoch 24 / 30, error: 0.19340000000000002
epoch 25 / 30, error: 0.18445999999999996
epoch 26 / 30, error: 0.19652000000000003
epoch 27 / 30,

In [118]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 80.434000


### Validate SVM on Fashion-MNIST

In [119]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 77.850000


### Test SVM on Fashion-MNIST

In [120]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 77.410000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [84]:
output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Rice

In [76]:
lr = 0.001
n_epochs = 50
reg_const = 0.05

svm_RICE = SVM(n_class_RICE, lr, n_epochs, reg_const)
svm_RICE.train(X_train_RICE, y_train_RICE)

epoch 1 / 50, train accuracy: 42.54422142791678
epoch 2 / 50, train accuracy: 63.88965264412061
epoch 3 / 50, train accuracy: 79.91934744753001
epoch 4 / 50, train accuracy: 75.0068737970855
epoch 5 / 50, train accuracy: 60.8193566125928
epoch 6 / 50, train accuracy: 82.00898176152506
epoch 7 / 50, train accuracy: 64.60452754101365
epoch 8 / 50, train accuracy: 79.40610393181193
epoch 9 / 50, train accuracy: 62.94565117771057
epoch 10 / 50, train accuracy: 62.82650536156172
epoch 11 / 50, train accuracy: 70.6443039134818
epoch 12 / 50, train accuracy: 76.10668133076712
epoch 13 / 50, train accuracy: 77.96718907524517
epoch 14 / 50, train accuracy: 83.08129410686463
epoch 15 / 50, train accuracy: 67.23490055906883
epoch 16 / 50, train accuracy: 88.46118595912382
epoch 17 / 50, train accuracy: 93.63944643020805
epoch 18 / 50, train accuracy: 89.91843094125194
epoch 19 / 50, train accuracy: 95.75657593254515
epoch 20 / 50, train accuracy: 98.9918430941252
epoch 21 / 50, train accuracy: 96

In [77]:
pred_svm = svm_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))

The training accuracy is given by: 99.908349


### Validate SVM on Rice

In [78]:
pred_svm = svm_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))

The validation accuracy is given by: 99.862524


## Test SVM on Rice

In [79]:
pred_svm = svm_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))

The testing accuracy is given by: 99.862524


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on Fashion-MNIST

In [ ]:
lr = 0.5
n_epochs = 10
reg_const = 0.5

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

In [ ]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

### Validate Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

### Testing Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Rice

In [ ]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
softmax_RICE.train(X_train_RICE, y_train_RICE)

In [ ]:
pred_softmax = softmax_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))

### Validate Softmax on Rice

In [ ]:
pred_softmax = softmax_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))

### Testing Softmax on Rice

In [ ]:
pred_softmax = softmax_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Training Logistic Classifer

In [ ]:
learning_rate = 0.5
n_epochs = 10
threshold = 0.5

lr = Logistic(learning_rate, n_epochs, threshold)
lr.train(X_train_RICE, y_train_RICE)

In [ ]:
pred_lr = lr.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))

### Validate Logistic Classifer

In [ ]:
pred_lr = lr.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))

### Test Logistic Classifier

In [ ]:
pred_lr = lr.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))